## Combining Stock Data With Reddit Posts 

In [66]:
import pandas as pd
import numpy as np
import datetime as dt
import re 
import string
import matplotlib.pyplot as plt

In [86]:
# Loading Stock data
apple_stock = pd.read_csv('HistoricalQuotes (2).csv')


In [4]:
# Cleaning Apple Stock Data Frame
apple_stock_clean = apple_stock
colstocheck = apple_stock_clean.columns
# Removes Money symbol
apple_stock_clean[colstocheck] = apple_stock_clean[colstocheck].replace({'\$':''}, regex = True)
# Removes white spaces
apple_stock_clean[colstocheck] = apple_stock_clean[colstocheck].replace({'\s':''}, regex = True)
apple_stock_clean.rename(columns={' Close/Last': 'Close', ' Open': 'Open'}, inplace = True)


In [5]:
apple_stock_clean['Close'] = apple_stock_clean['Close'].astype(float)
apple_stock_clean['Open'] = apple_stock_clean['Open'].astype(float)

# Creating a column that outputs the change in stock price
diff_df_stock = apple_stock_clean
diff_df_stock['Day_Diff'] = diff_df_stock.Close - diff_df_stock.Open

In [6]:
#making class for increase or decrease in stock 
change = diff_df_stock['Day_Diff'].to_list()

up_down = []
for stock_change in change:
    if stock_change > 0:
        up_downs = 0
    elif stock_change < 0:
        up_downs = 1
        
    up_down.append(up_downs)
    
diff_df_stock['Class'] = up_down

In [7]:
def date_convert(dataframe): 

    """
    This function takes the date from the stock dataframe and moves it back a day to match reddit data and date

    """
    dates_list = dataframe.Date.tolist()
    new_dates = []
    for date in dates_list:
        new_date = ''
        month = date[:2]
        day = date[3:5]
        year = date[6:]

        u = (int(day)-1)
        if u < 10 and u > 0:
            day_c = '0'+ str(u)
        elif u == 0 and month == '02':
            day_c = '31'
            month = '01'
        elif u == 0 and month == '03':
            day_c = '28'
            month = '02'
        elif u == 0 and month == '04':
            day_c = '31'
            month = '03'
        elif u == 0 and month == '05':
            day_c = '30'
            month = '04'
        elif u == 0 and month == '06':
            day_c = '31'
            month = '05'
        elif u == 0 and month == '07':
            day_c = '30'
            month = '06'
        elif u == 0 and month == '08':
            day_c = '31'
            month = '07'
        elif u == 0 and month == '09':
            day_c = '31'
            month = '08'
        elif u == 0 and month == '10':
            day_c = '30'
            month = '09'
        elif u == 0 and month == '11':
            day_c = '31'
            month = '10'
        elif u == 0 and month == '12':
            day_c = '30'
            month = '11'
        else:
            day_c = str(u)

        new_date = year+'-'+ month +'-'+ day_c
        new_dates.append(new_date)
        
    return new_dates

In [8]:
converted_dates = date_convert(diff_df_stock)

In [9]:
# reads in apple data from Topic Modeling notebook
df_apple1 = pd.read_csv('Apple_topics_df.csv')

In [10]:
# Removes fridays and saturdays since the stock market isnt active on saturdays or sundays
apple_df_days_removed = df_apple1[df_apple1['Day_of_Week'] != 4]
apple_df_days_removed = apple_df_days_removed[apple_df_days_removed['Day_of_Week'] != 5]
apple_df_days_removed.reset_index(inplace = True, drop = True)

In [12]:
# Finds overapping dates between stock market df and apple dataframe 
date_app = apple_df_days_removed.Date_Posted.unique().tolist()

is_in = []
index = []
for count, date in enumerate(converted_dates):
    if date in date_app:
        index.append(count)
        is_in.append(date)

In [13]:
# sorts out dates not present in stock market dataframe
date_sorted =[]
for ind in index:
    entry = diff_df_stock.iloc[ind].tolist()
    date_sorted.append(entry)

In [63]:
# create new dataframe for stock data
apple_stock_sort = pd.DataFrame(date_sorted, columns = ['Date','Close', 'Volume','Open', 'High','Low','Day_Diff','Class'] )

In [15]:
# the length of the stock dataframe has less dates than the reddit dataframe, mostly due to holidays
# converting teh dates again will allow us to match each date up perfectly 
converted_dates2 = date_convert(apple_stock_sort)

In [16]:
# finds dates not in stock df
date_app = apple_df_days_removed.Date_Posted.unique().tolist()

not_in = []
for date in date_app:
    if date not in converted_dates2:
        not_in.append(date)

In [17]:
# removes rows with dates that dont match stock data
for count, entry in enumerate(not_in):
    if count == 0:
        date_sorted2 = apple_df_days_removed[apple_df_days_removed['Date_Posted'] != entry]
    else:
        date_sorted2 = date_sorted2[date_sorted2['Date_Posted'] != entry]

In [54]:
# This takes reddit data and puts all posts and their information from one day into a singular list
# This is done so that teh information can be inserted into a machine learning model 
dates = date_sorted2.Date_Posted.unique().tolist()

date_list = []
for date in dates:
    dates = apple_df_days_removed.groupby('Date_Posted').get_group(date)
    dates_lists = []
    nc = dates.Number_of_Comments.tolist()
    ur = dates.Upvote_ratio.tolist()
    s = dates.Score.tolist()
    t = dates.Topic.tolist()
    topic_list = []
    # breaks topics into vectors so it is easier to find each topics feature importance later 
    for i ,j in enumerate(t):
        zeros = np.zeros(len(date_sorted2.Topic.unique())).tolist()
        zeros = [round(x) for x in zeros] # change zeros to int
        zeros[j] = 1
        topic_list.append(zeros)
    topic_list = [inner for outer in topic_list for inner in outer] # combine lists in the list
    i = dates.Total_Interactions
    #dates_lists = [t, ur, i, nc, s]
    dates_lists = [topic_list,ur,i,nc,s]
    
    compiled_info = [j for i in dates_lists for j in i] # combine lists in the list
    
    # Creates list that can be fed into machine learning models
    date_list.append(compiled_info)

In [55]:
# adding stock change to above list
apple_final_df = pd.DataFrame(date_list)
apple_final_df['Stock_Change'] = apple_stock_sort.Class.tolist()

In [56]:
# Distribution of increase or deacrease in stock data
apple_final_df['Stock_Change'].value_counts()

0    450
1    371
Name: Stock_Change, dtype: int64

In [58]:
# creates csv for next notebook
apple_final_df.to_csv('Apple_stocks_data_test.csv', index= False)